In [1]:
import os
import torch
import torchaudio
import torchaudio.transforms as T
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import nemo.collections.asr as nemo_asr
from torch.optim import AdamW
# from torch.cuda.amp import GradScaler, autocast

[NeMo W 2024-10-11 17:14:47 nemo_logging:349] /home/keagan/anaconda3/envs/vc/lib/python3.10/site-packages/megatron/core/tensor_parallel/layers.py:254: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
      def forward(
    
[NeMo W 2024-10-11 17:14:47 nemo_logging:349] /home/keagan/anaconda3/envs/vc/lib/python3.10/site-packages/megatron/core/tensor_parallel/layers.py:265: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
      def backward(ctx, grad_output):
    
[NeMo W 2024-10-11 17:14:47 nemo_logging:349] /home/keagan/anaconda3/envs/vc/lib/python3.10/site-packages/megatron/core/tensor_parallel/layers.py:325: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
      def forward(
    
[NeMo W 2024-10-11 17:14:47 nemo_loggin

## DAtaset

In [2]:
class LibriTTSDataset(Dataset):
    def __init__(self, root_dir, sample_rate=22050, n_mels=80, n_fft=1024, hop_length=256, win_length=1024,target_duration = 5.0):
        """
        Args:
            root_dir (str): Path to the root directory of the LibriTTS dataset.
            sample_rate (int): Sampling rate for audio files.
            n_mels (int): Number of mel bands.
            n_fft (int): FFT size for mel spectrogram.
            hop_length (int): Hop length for mel spectrogram.
            win_length (int): Window length for mel spectrogram.
        """
        self.root_dir = root_dir
        self.sample_rate = sample_rate
        self.target_duration = target_duration

        # Initialize the MelSpectrogram transformation
        self.mel_transform = T.MelSpectrogram(
            sample_rate=sample_rate,
            n_mels=n_mels,
            n_fft=n_fft,
            hop_length=hop_length,
            win_length=win_length,
        )

        # Collect all audio file paths and their corresponding labels (speaker IDs)
        self.audio_files, self.labels, self.speaker_to_label = self._load_audio_paths_and_labels()

    def _load_audio_paths_and_labels(self):
        """Load all audio file paths and assign a label to each speaker folder."""
        audio_files = []
        labels = []
        speaker_to_label = {}
        speaker_id = 0  # Start labeling speakers from 0

        # Assign a unique label to each speaker folder
        for speaker_folder in os.listdir(self.root_dir):
            speaker_path = os.path.join(self.root_dir, speaker_folder)
            if os.path.isdir(speaker_path):
                # Map speaker folder to a label
                if speaker_folder not in speaker_to_label:
                    speaker_to_label[speaker_folder] = speaker_id
                    speaker_id += 1

                # Traverse subfolders and find audio files
                for subfolder in os.listdir(speaker_path):
                    subfolder_path = os.path.join(speaker_path, subfolder)
                    if os.path.isdir(subfolder_path):
                        for file in os.listdir(subfolder_path):
                            if file.endswith(".wav"):  # Assuming .wav audio format
                                audio_files.append(os.path.join(subfolder_path, file))
                                labels.append(speaker_to_label[speaker_folder])  # Assign the speaker's label

        return audio_files, labels, speaker_to_label

    def __len__(self):
        return len(self.audio_files)

    def __getitem__(self, idx):
        # Load audio
        audio_path = self.audio_files[idx]
        label = self.labels[idx]
        # speaker_to_label = self.speaker_to_label[idx]
        waveform, sr = torchaudio.load(audio_path)

        # Resample audio if necessary
        if sr != self.sample_rate:
            resample_transform = T.Resample(orig_freq=sr, new_freq=self.sample_rate)
            waveform = resample_transform(waveform)

        target_length = int(self.target_duration * self.sample_rate)
        audio_length = waveform.size(1)
        
        # Case 1: Trim the audio if it's too long
        if audio_length > target_length:
            waveform = waveform[:, :target_length]
        
        # Case 2: Pad the audio if it's too short
        elif audio_length < target_length:
            padding = target_length - audio_length
            waveform = torch.nn.functional.pad(waveform, (0, padding))
        
        # return waveform

        # Convert to mel spectrogram
        mel_spectrogram = self.mel_transform(waveform)
        mel_spectrogram = mel_spectrogram.squeeze()
        # print(f"speaker to label {mel_spectrogram.shape}")

        # Return the mel spectrogram and the corresponding speaker label
        return mel_spectrogram, label

In [3]:
# train_data_path = "/home/keagan/Documents/projects/SelfVC/data/archive/"
# batch_size = 32

In [4]:
# train_dataset = LibriTTSDataset(train_data_path)
# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)


In [5]:
# mel_spec = train_dataset[5]

In [6]:
import torch
import torchaudio.transforms as T
from torch.nn.utils.rnn import pad_sequence


def collate_fn(batch):
    # Separate inputs and targets
    inputs, targets = zip(*batch)
    
    # Pad sequences in the batch to the maximum length
    inputs_padded = pad_sequence(inputs, batch_first=True, padding_value=0)
    targets_padded = pad_sequence(targets, batch_first=True, padding_value=-1)
    
    return inputs_padded, targets_padded

# def pad_collate_fn(batch):
#     """
#     Pads the Mel spectrograms in a batch to have the same time dimension.
#     """
#     # Get the maximum length of the time dimension in the batch
#     max_len = max([mel_spec[0].shape[2] for mel_spec in batch])

#     # Pad each mel spectrogram to the max length in the batch
#     padded_batch = []
#     label_batch = []
#     for mel_spec in batch:
#         # Pad along the time dimension (dim=2)
#         pad_amount = max_len - mel_spec[0].shape[2]
#         padded_mel_spec = torch.nn.functional.pad(mel_spec[0], (0, pad_amount), mode='constant', value=0)
#         padded_mel_spec = padded_mel_spec.squeeze()
#         padded_batch.append(padded_mel_spec)
#         label_batch.append(mel_spec[1])


#     # Stack the batch into a tensor
#     return torch.stack(padded_batch),torch.Tensor(label_batch)

# def collate_fn(batch):
#     mel_specs, labels = zip(*batch)
#     print(mel_specs)
    
#     # Find the max time dimension for padding
#     max_len = max([mel_specs.shape[2] for mel_spec in batch])
    
#     # Pad all Mel spectrograms to the same length
#     mel_specs_padded = [torch.nn.functional.pad(mel, (0, max_len - mel.shape[2])) for mel in mel_specs]
    
#     mel_specs_padded = torch.stack(mel_specs_padded)  # (batch_size, n_mels, max_len)
#     labels = torch.tensor(labels)  # (batch_size,)
    
#     return mel_specs_padded, labels

In [7]:
# libri_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=pad_collate_fn)

# # Test the DataLoader again
# for i, mel_spectrogram_batch in enumerate(libri_dataloader):
#     print(f"Batch {i+1}")
#     # print(f"Mel Spectrogram Batch Shape: {mel_spectrogram_batch[1]}")
#     print(mel_spectrogram_batch[1].shape)
    
#     # Break after a few batches to avoid printing everything
#     if i == 2:
#         break

## Utils

In [8]:
import random

def generate_pairs_with_labels(embeddings, speaker_labels):
    """
    Generate positive and negative pairs for contrastive learning based on speaker labels.
    
    Args:
        embeddings (torch.Tensor): The feature embeddings of shape (batch_size, embedding_dim).
        speaker_labels (torch.Tensor): Tensor of speaker labels of shape (batch_size).
        
    Returns:
        tuple of (positive_pairs, negative_pairs): Indices for positive and negative pairs.
    """
    positive_pairs = []
    negative_pairs = []
    
    # Iterate through each possible pair in the batch
    for i in range(len(speaker_labels)):
        for j in range(i + 1, len(speaker_labels)):
            if speaker_labels[i] == speaker_labels[j]:
                # Positive pair (same speaker)
                positive_pairs.append((i, j))
            else:
                # Negative pair (different speakers)
                negative_pairs.append((i, j))
    
    return positive_pairs, negative_pairs


In [9]:

import torch
import torch.nn.functional as F

def compute_contrastive_loss(embeddings, positive_pairs, negative_pairs, margin=1.0):
    """
    Compute contrastive loss given the embeddings, positive pairs, and negative pairs.
    
    Args:
        embeddings (torch.Tensor): The feature embeddings of shape (batch_size, embedding_dim).
        positive_pairs (list): List of tuples containing indices of positive pairs.
        negative_pairs (list): List of tuples containing indices of negative pairs.
        margin (float): The margin for contrastive loss.
    
    Returns:
        torch.Tensor: The computed contrastive loss.
    """
    loss = 0.0
    for (i, j) in positive_pairs:
        # Positive pairs: minimize the distance
        distance = torch.nn.functional.pairwise_distance(embeddings[i], embeddings[j])
        loss += distance.pow(2)  # Minimize positive distance (same speaker)

    for (i, j) in negative_pairs:
        # Negative pairs: maximize the distance up to the margin
        distance = torch.nn.functional.pairwise_distance(embeddings[i], embeddings[j])
        loss += torch.relu(margin - distance).pow(2)  # Maximize negative distance (different speakers)

    return loss / (len(positive_pairs) + len(negative_pairs))


In [10]:

# for i, mel_spectrogram_batch in enumerate(libri_dataloader):
#     print(f"Batch {i+1}")
#     embeddings = torch.randn(64, 256)
#     # print(f"Mel Spectrogram Batch Shape: {mel_spectrogram_batch[1]}")
#     # print(mel_spectrogram_batch[1].shape)
#     positive_pairs, negative_pairs = generate_pairs_with_labels(embeddings, mel_spectrogram_batch[1])
#     print(len(positive_pairs),len(negative_pairs))
#     loss = compute_contrastive_loss(embeddings, positive_pairs, negative_pairs, margin=1.0)
#     print(loss)
#     # Break after a few batches to avoid printing everything
#     if i == 2:
#         break



In [11]:
# def create_masked_indices(sequence_length, mask_prob=0.15):
#     """
#     Create masked indices for the MLM task.
    
#     Args:
#         sequence_length (int): Length of the sequence to mask.
#         mask_prob (float): Probability of masking a position.
    
#     Returns:
#         torch.Tensor: A tensor of masked indices (1 for masked, 0 for unmasked).
#     """
#     # Generate a mask of the same length as the sequence with 1 for masked positions
#     mask = torch.bernoulli(torch.full((sequence_length,), mask_prob)).bool()
#     return mask

In [12]:
# def apply_mask_to_embeddings(embeddings, mask_indices, mask_token=None):
#     """
#     Apply masking to the embeddings based on the mask indices.
    
#     Args:
#         embeddings (torch.Tensor): The input embeddings (batch_size, sequence_length, embedding_dim).
#         mask_indices (torch.Tensor): Boolean mask indicating which positions to mask.
#         mask_token (torch.Tensor): The token to replace masked positions (default: zero tensor).
    
#     Returns:
#         torch.Tensor: Masked embeddings.
#     """
#     if mask_token is None:
#         # Default mask token is a tensor of zeros with the same dimension as each embedding
#         mask_token = torch.zeros(embeddings.size(-1))

#     # Clone the embeddings to avoid modifying the original
#     masked_embeddings =                     
#     # Apply mask token to the masked positions
#     masked_embeddings[mask_indices] = mask_token
    


#     return masked_embeddings

In [13]:
def create_masked_indices(sequence_length, mask_prob=0.15):
    # Generate random mask indices for the sequence length
    mask = torch.rand(sequence_length) < mask_prob
    return mask

def apply_mask_to_embeddings(mel_specs, mask_indices, mask_token_value=0.0):
    """
    Apply a mask to the mel spectrogram embeddings across the sequence length for each sample in the batch.
    """
    # mel_specs: [batch_size, num_mel_bands, sequence_length]
    batch_size, num_mel_bands, sequence_length = mel_specs.shape

    # Expand the mask for the batch size and mel bands
    mask_indices_expanded = mask_indices.unsqueeze(0).unsqueeze(1).expand(batch_size, num_mel_bands, -1)
    
    # Clone the mel_specs and apply the mask token value
    masked_mel_specs = mel_specs.clone()
    masked_mel_specs[mask_indices_expanded] = mask_token_value
    
    return masked_mel_specs

In [14]:
def masked_language_modeling_loss( masked_mel_specs,original_embeddings, model, mask_indices,lengths):
    """
    Compute MLM loss for audio embeddings.
    
    Args:
        original_embeddings (torch.Tensor): The original unmasked embeddings (batch_size, sequence_length, embedding_dim).
        masked_embeddings (torch.Tensor): The masked embeddings (batch_size, sequence_length, embedding_dim).
        model (nn.Module): The model to predict masked embeddings.
        mask_indices (torch.Tensor): Boolean mask indicating which positions were masked.
    
    Returns:
        torch.Tensor: The computed MLM loss.
    """
    # Pass the masked embeddings through the model
    # output_embeddings = model(masked_embeddings)
    # lengths = torch.full((masked_mel_specs.shape[0],), masked_mel_specs.shape[2], dtype=torch.int32).cuda()
    # output_embeddings, output_embeddings_length = model(
    # audio_signal=masked_mel_specs,
    # length=lengths
    # )
    model.eval()
    with torch.no_grad():  # Disable gradient calculation for inference
        output_embeddings, output_embeddings_length = model(
            audio_signal=masked_mel_specs,
            length=lengths
        )
    # Compute loss only on masked positions
    # masked_original = original_embeddings[mask_indices]
    # masked_output = output_embeddings[mask_indices]
    
    # Mean squared error for regression
    mlm_loss = torch.nn.functional.mse_loss(output_embeddings, original_embeddings)
    
    return mlm_loss

# Model

In [15]:
# device = torch.device("cuda")
asr_model = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="nvidia/stt_en_conformer_ctc_large")
conformer_encoder = asr_model.encoder.cuda()


[NeMo I 2024-10-11 17:14:52 mixins:172] Tokenizer SentencePieceTokenizer initialized with 128 tokens


[NeMo W 2024-10-11 17:14:52 modelPT:165] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data/NeMo_ASR_SET/English/v2.0/train/tarred_audio_manifest.json
    sample_rate: 16000
    batch_size: 32
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    shuffle_n: 2048
    is_tarred: true
    tarred_audio_filepaths: /data/NeMo_ASR_SET/English/v2.0/train/audio__OP_0..4095_CL_.tar
    
[NeMo W 2024-10-11 17:14:52 modelPT:172] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath:
    - /data/ASR/LibriSpeech/librispeech_withs

[NeMo I 2024-10-11 17:14:52 features:289] PADDING: 0


[NeMo W 2024-10-11 17:14:53 nemo_logging:349] /home/keagan/anaconda3/envs/vc/lib/python3.10/site-packages/nemo/core/connectors/save_restore_connector.py:571: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related t

[NeMo I 2024-10-11 17:14:53 save_restore_connector:249] Model EncDecCTCModelBPE was successfully restored from /home/keagan/.cache/huggingface/hub/models--nvidia--stt_en_conformer_ctc_large/snapshots/2c8326e4e43ae5b994612cfea3f3029818fb23c6/stt_en_conformer_ctc_large.nemo.


# Fine Tune

In [16]:
batch_size = 32
learning_rate = 5e-5
data_dir = '/home/keagan/Documents/projects/SelfVC/data/archive/'
model_save_path = '/home/keagan/Documents/projects/SelfVC/models/conformer_encoder_v1.pth'
log_file_path = '/home/keagan/Documents/projects/SelfVC/models/conformer_encoder_loss.txt'
optimizer = AdamW(conformer_encoder.parameters(), lr=learning_rate, betas=(0.9, 0.99))
epochs = 25


In [17]:
train_dataset = LibriTTSDataset(data_dir)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# collate_fn=collate_fn
# collate_fn=pad_collate_fn


In [18]:
# scaler = GradScaler()

In [19]:
for epoch in range(epochs):  # Train for 50 epochs
    conformer_encoder.train()
    total_loss = 0
    for mel_specs, labels in train_loader:
        optimizer.zero_grad()
        mel_specs = mel_specs.cuda()  # (batch_size, n_mels, max_len)
        labels = labels  # (batch_size,)
        
        # Forward pass through the model
        # embeddings = conformer_encoder(mel_specs)  # (batch_size, output_dim, max_len)
        
        # print(mel_specs.shape[2])
        # print(torch.tensor(mel_specs.shape[2]))
        lengths = torch.full((mel_specs.shape[0],), mel_specs.shape[2], dtype=torch.int32).cuda()
        # print(lengths.shape)
        # with autocast():
        embeddings, embeddings_length = conformer_encoder(
            audio_signal=mel_specs,
            length=lengths
        )
        # print("Test")
        # Contrastive loss
        positive_pairs, negative_pairs = generate_pairs_with_labels(embeddings, labels)
        contrastive_loss = compute_contrastive_loss(embeddings, positive_pairs, negative_pairs)
        contrastive_loss = contrastive_loss.mean()
        # MLM loss (masked positions)
        mask_indices = create_masked_indices(sequence_length=mel_specs.shape[2])
        masked_mel_specs = apply_mask_to_embeddings(mel_specs, mask_indices)
        # masked_mel_specs = apply_mask_to_embeddings(mel_specs, mask_indices)
        mlm_loss = masked_language_modeling_loss(masked_mel_specs, embeddings,conformer_encoder, mask_indices,lengths)
        
        # Combine losses
        loss = contrastive_loss + mlm_loss
        total_loss += loss.item()
        # print(contrastive_loss,mlm_loss)
        with open(log_file_path, 'a') as log_file:
            log_file.write(f'Contrastive Loss: {contrastive_loss:.6f} MLM Loss: {mlm_loss:.6f}\n')

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        # scaler.scale(loss).backward()
        # scaler.step(optimizer)
        # scaler.update()        
        torch.cuda.empty_cache()
    
    print(f'Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}')
    with open(log_file_path, 'a') as log_file:
        log_file.write(f'Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}\n')    
    torch.save(conformer_encoder, model_save_path)

KeyboardInterrupt: 